In [ ]:
import zipfile
file=zipfile.ZipFile('trainingandtestdata.zip')
file.extractall()

In [1]:
import pandas as pd
import numpy as np

columns=['Polarity', 'ID', 'Date', 'Query', 'User', 'Text']

df_train = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='iso-8859-1', header=None, names=columns)
df_test = pd.read_csv('testdata.manual.2009.06.14.csv', encoding='iso-8859-1', header=None, names=columns)

# df_train.columns=columns

In [2]:
df_train.head()
#df_test.head()

,Polarity,ID,Date,Query,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
np.unique(df_test['Query'])
df_test[df_test.Query == 'NO_QUERY']

,Polarity,ID,Date,Query,User,Text


In [4]:
df_train.isnull().sum()

Polarity    0
ID          0
Date        0
Query       0
User        0
Text        0
dtype: int64

In [5]:
np.unique(df_train['Polarity'])

array([0, 4], dtype=int64)

In [6]:
df_train.shape

(1600000, 6)

In [7]:
df_train[df_train.Query != 'NO_QUERY']

,Polarity,ID,Date,Query,User,Text


In [8]:
#df_train['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
#'Mon Apr 06 22:19:45 PDT 2009'
# '%a %b %d %H:%M:%S %z %y'
df_train['DateTime']=pd.to_datetime(df_train['Date'], format='%a %b %d %H:%M:%S PDT %Y')

In [ ]:
#X_train=df_train['Text']
#y_train=df_train['Polarity']
#X_test=df_test['Text']
#y_test=df_test['Polarity']

In [9]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:8: SyntaxWarning: invalid escape sequence '\W'
<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:8: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Shabnam\AppData\Local\Temp\ipykernel_14216\2915367723.py:7: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
C:\Users\Shabnam\AppData\Local\Temp\ipykernel_14216\2915367723.py:8: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')


In [10]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
# {'perceptron', 'log_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive', 'squared_error', 'hinge', 'squared_hinge', 'modified_huber'}
clf = SGDClassifier(loss='log_loss', random_state=1)
#doc_stream = stream_docs(path='movie_data.csv')

In [11]:
import pyprind
pbar = pyprind.ProgBar(1600)
classes = np.array([0, 4])
for i in range(1600):
    #print(1000*i,1000*(i+1))
    df_train_i=df_train.sample(1000, replace=False)
    df_train.drop(df_train_i.index, inplace=True)
    X_train_i, y_train_i=df_train_i['Text'], df_train_i['Polarity']
    #X_train_i, y_train_i = X_train.iloc[1000*i:1000*(i+1)],y_train.iloc[1000*i:1000*(i+1)] #get_minibatch(doc_stream, size=1000)
    X_train_i = vect.transform(X_train_i)
    clf.partial_fit(X_train_i, y_train_i.values, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:12:20


In [12]:
new_df_test=df_test[df_test['Polarity']!=2]
X_test=new_df_test['Text']
y_test=new_df_test['Polarity']

X_test = vect.transform(X_test)
print('Accuracy: %.5f' % clf.score(X_test, y_test.values))

Accuracy: 0.77437


- squared_hinge Accuracy: 0.50696
- modified_huber Accuracy: 0.50696
- perceptron Accuracy: 0.50696
- log_loss 0.50696
- huber Accuracy: 0.50696
- epsilon_insensitive Accuracy: 0.50696
- squared_epsilon_insensitive Accuracy: 0.49582
- squared_error Accuracy: 0.47075
- hinge Accuracy: 0.50696

In [ ]:
print(np.unique(df_test['Polarity']))
print(df_test.shape)
print(df_test[df_test['Polarity']==2].shape)
print(np.unique(y_train))

In [ ]:
print(df_test[df_test['Polarity']==2].shape)
print(df_test[df_test['Polarity']!=2].shape)
print(df_test.shape)
print(df_test[df_test['Polarity']==4].shape)



In [ ]:
tokenized_X_train=X_train.apply(tokenizer)

In [ ]:
print(tokenized_X_train[0])
print(X_train[0])
y_pred = clf.predict(X_test)

In [ ]:
(y_pred != y_test).sum()

In [ ]:
y_test.values

In [ ]:
print(y_train[y_train==0].shape)
print(y_train[y_train==4].shape)
print(y_test[y_test==0].shape)
print(y_test[y_test==4].shape)



In [ ]:
print(X_train.shape)
X_train=df_train.sample(1000, replace=False)
df_train.drop(X_train.index)
print(X_train.shape)
